## Introduction to Craters on Planetary Surfaces

#### Learning objectives
_Skills_
* Use of LandLab model grids, with the possibility of running many other LandLab components to route any kind of flow over a cratered surface, or vary erosion during cratering
* Make changes to input parameters to vary the possible sizes of craters 
* Gain hands-on experience with visualizing output in Python

_Key concepts_
* Explore and understand the process of simulation of craters on a landscape
* Explore how an initially cratered landscape is affected by hillslope diffusion and the flow of surface water

In [1]:
import numpy as np
from landlab import RasterModelGrid, imshow_grid, NodeStatus, values
import random as rdm